# Deep Q Learning (with Open AI GYM)
### GYM:
Open AI의 Gym은 강화학습을 위한 개발 및 평가 환경을 제공하는 오픈 소스 라이브러리이다. 다양한 강화학습 문제를 다룰 수 잇는 표준화된 인터페이스와 환경을 제공한다. Gym을 통해 에이전트가 환경과 상호작용하며 상태를 관찰하고, 행동을 선택하고, 보상을 받으며, 학습을 진행한다. 

In [1]:
# Import Lib
import gym 
import collections  # 추가적인 데이터 구조를 위한 lib (deque)
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# Define Hyperparameters
learning_rate = 0.0005  # 한 번에 학습을 얼마나 할 것인지
gamma         = 0.98    # 감쇠율
buffer_limit  = 50000   # 버퍼에 저장할 수 잇는 최대 경험의 개수
batch_size    = 32      # 기댓값을 구현하기 위한 미니 데이터 집합

In [3]:
# Class Replay Buffer() - off-policy 인 DQN에서 사용
# 경험을 저장하고 샘플링하여 네트워크를 학습함
class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit) # maxlen이 buffer limit크기인 deque 객체 생성(FIFO) 
    
    def put(self, transition):
        self.buffer.append(transition) # 주어진 transition을 Replay Buffer에 추가
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n) # 주어진 n만큽의 랜덤한 경험을 샘플링
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], [] 
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition #경험의 상태, 행동, 보상, 다음 상태, 종료 여부 리스트에 저장
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst) # torch.tensor 형태로 변환후 return
    
    def size(self):
        return len(self.buffer)

In [4]:
# Qnet 정의 
# State를 입력으로 받아, 가능한 Action에 대한 Q값 출력 
# 주어진 State에서 최적의 Action을 선택할 수 있도록 하는 DQN 알고리즘
class Qnet(nn.Module):
    def __init__(self): # 신경망 구조 정의
        super(Qnet, self).__init__() 
        self.fc1 = nn.Linear(4, 128) # 입력 크기 4, 출력 128 
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x): # 신경망의 순전파 연산 수행
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        x = self.fc3(x) 
        return x # Q값 출력
      
    def sample_action(self, obs, epsilon): #주어진 state =obs와 e-greedy정책에 따라 행동하게끔 설정
        out = self.forward(obs) 
        coin = random.random()
        if coin < epsilon: #epsilon확률로 무작위 행동을 선택하고 1-epsilon 확률로 Q값이 가장 큰 행동 선택한다.
            return random.randint(0,1) 
        else : 
            return out.argmax().item()

In [5]:
# Qnet을 학습하는 함수
# s와 s_prime을 통해 target_q 계산, gradient 계산 후 q-net 학습
def train(q, q_target, memory, optimizer): 
    # Parameter: q-Qnet모델, q_target-targetQ-net, memory-메모리버퍼,optimizer-optimizer
    for i in range(10): # 메모리 버퍼에서 배치 크기 만큼 샘플 추출
        s,a,r,s_prime,done_mask = memory.sample(batch_size)

        q_out = q(s)# s에 대한 Q값
        q_a = q_out.gather(1,a) # 실제 수행한 a에 대한 Q값
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1) #타겟 qnet모델을 통해. 다음 상태 s_prime 에 대한 최대 Q값 계산
        target = r + gamma * max_q_prime * done_mask # target Q값 계산 공식
        loss = F.smooth_l1_loss(q_a, target) #huber loss함수를 사용해 예측 오차 계산
        
        optimizer.zero_grad() #optimizer gradient 값 초기화
        loss.backward() # 손실에 대한 역전파 수행, gradient 계산
        optimizer.step()# optimizer parameter 업데이트

In [ ]:
# CartPole 환경에서 DQN 적용
# 
def main():
    env = gym.make('CartPole-v1')
    q = Qnet()#qnet 모델 생성
    q_target = Qnet()#target-qnet 생성
    q_target.load_state_dict(q.state_dict())#target qnet 모델의 초기 parameter를 현재 qnet 모델의 parameter로 설정
    memory = ReplayBuffer() # Replaybuffer 활용해 memory 버퍼 생성

    print_interval = 20 #일정 episode마다 결과를 출력할 주기 설정
    score = 0.0  # 누적 보상 저장 변수
    optimizer = optim.Adam(q.parameters(), lr=learning_rate) # Adam optimizer 활용해 qnet 학습

    for n_epi in range(10000): #총 10000번 에피소드동안 학습
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #epsilon 설정 anealing from 8% to 1% 
        s, _ = env.reset() #환경 초기화하고 초기상태 s를 얻는다
        done = False

        while not done: #에피소드가 종료될때까지 반복
            a = q.sampel_action(torch.from_numpy(s).float(), epsilon) #qnet모델을 사용해 상태 s에 대한 액션 a를 선택
            s_prime, r, done, truncated, info = env.step(a)# 선택한 액션을 환경에 적용해 s_prime과 보상 종료여부 반환
            done_mask = 0.0 if done else 1.0 #종료 여부에따라 마스크 설정
            memory.put((s,a,r/100.0,s_prime, done_mask)) #메모리 버퍼에 현재 상태,행동, 보상, 완료 마스크 저장
            s = s_prime # 다음상태를 현재 상태로 업데이트

            score += r # 누적보상 계산
            if done:
                break 
            
        if memory.size()>2000: # 메모리버퍼의 사이즈가 2000이상일때 학습
            train(q, q_target, memory, optimizer) 

        if n_epi%print_interval==0 and n_epi!=0: #일정 주기마다 학습 결과 출력
            q_target.load_state_dict(q.state_dict()) #q_net 모델의 파라미터 업데이트 
            # 에피소드 결과값 출력 
            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                            n_epi, score/print_interval, memory.size(), epsilon*100))
            score = 0.0
    env.close() # 모든 알고리즘 종료시 환경 종료

if __name__ == '__main__':
    main()

C:\Users\AIMLAB_JKIM\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


n_episode :20, score : 40.5, n_buffer : 809, eps : 7.9%
n_episode :40, score : 35.4, n_buffer : 1516, eps : 7.8%


C:\Users\AIMLAB_JKIM\AppData\Local\Temp\ipykernel_12924\1099629588.py:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \


n_episode :60, score : 28.1, n_buffer : 2079, eps : 7.7%
n_episode :80, score : 9.9, n_buffer : 2277, eps : 7.6%
n_episode :100, score : 12.4, n_buffer : 2525, eps : 7.5%
n_episode :120, score : 12.9, n_buffer : 2783, eps : 7.4%
n_episode :140, score : 33.9, n_buffer : 3461, eps : 7.3%
n_episode :160, score : 25.2, n_buffer : 3966, eps : 7.2%
n_episode :180, score : 59.0, n_buffer : 5145, eps : 7.1%
n_episode :200, score : 134.2, n_buffer : 7829, eps : 7.0%
n_episode :220, score : 199.2, n_buffer : 11812, eps : 6.9%
n_episode :240, score : 191.2, n_buffer : 15637, eps : 6.8%
n_episode :260, score : 201.8, n_buffer : 19673, eps : 6.7%
n_episode :280, score : 193.0, n_buffer : 23533, eps : 6.6%
n_episode :300, score : 151.7, n_buffer : 26567, eps : 6.5%
n_episode :320, score : 184.9, n_buffer : 30265, eps : 6.4%
n_episode :340, score : 199.9, n_buffer : 34263, eps : 6.3%
n_episode :360, score : 199.4, n_buffer : 38252, eps : 6.2%
n_episode :380, score : 221.1, n_buffer : 42674, eps : 6.1